In [1]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.params = []
        
    def forward(self,x):
        return 1/(1+np.exp(-1))

In [2]:
class Affine:
    def __init__(self,W,b):
        self.params = [W,b]
    
    def forward(self,x):
        W, b = self.params
        out = np.dot(x,W)+b
        return out

In [6]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 重みとバイアスの初期化
        W1 = np.random.randn(I,H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H,O)
        b2 = np.random.randn(O)
        
        # レイヤの生成
        self.layers = [
            Affine(W1,b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        
        # 全ての重みをリストにまとめる
        self.params = []
        for layer in self.layers:
            self.params += layer.params
        
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
            return x

In [8]:
x = np.random.randn(10,2)
model = TwoLayerNet(2,4,3)
s = model.predict(x)

In [9]:
print(s)

[[ 1.80594621  1.09722484  0.70259079 -3.12833094]
 [-1.66902612  0.31643616  0.19164814 -0.13929605]
 [ 0.42757856  7.09569264 -5.66773862 -1.43486839]
 [-1.80482323 -0.38128375  0.8240273  -0.07620283]
 [-0.05232176  1.13252473 -0.01338569 -1.49346605]
 [ 1.60842447 -1.32834962  3.00170188 -3.15013048]
 [-0.28040062  1.42083119 -0.37891122 -1.26994554]
 [ 1.08550241 -1.15270167  2.63820074 -2.67673373]
 [-0.86064352  3.44127009 -2.56713229 -0.5976903 ]
 [-0.29916606  1.39420667 -0.3597614  -1.25560822]]
